In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from citipy import citipy
import requests
from config import weather_api_key


In [12]:
# Create a set of random lat and long combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
longs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_longs = zip(lats, longs)

In [13]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_longs)

In [14]:
# create a list for holding the cities.
cities = []
#identify the nearest city for each lat long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    #if the city is unique then we will at it to the cities list.
    if city not in cities:
        cities.append(city)
#Print the city count to confirm sufficient count.
len(cities)

762

In [15]:
# Starting URL for Weather Map API Call.(from module)
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [16]:
# create an empty list to hold weather data
city_data = []
#print the beginning of the loging.
print("Beginning Data Retrieval    ")
print("----------------------------")

#create counter
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | komsomolskiy
Processing Record 2 of Set 1 | hilo
Processing Record 3 of Set 1 | fortuna
Processing Record 4 of Set 1 | thompson
Processing Record 5 of Set 1 | meulaboh
Processing Record 6 of Set 1 | gweta
Processing Record 7 of Set 1 | hailey
Processing Record 8 of Set 1 | punta arenas
Processing Record 9 of Set 1 | saint-augustin
Processing Record 10 of Set 1 | ixtapa
Processing Record 11 of Set 1 | sataua
City not found. Skipping...
Processing Record 12 of Set 1 | cidreira
Processing Record 13 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 14 of Set 1 | castro
Processing Record 15 of Set 1 | bambous virieux
Processing Record 16 of Set 1 | pangnirtung
Processing Record 17 of Set 1 | puerto ayora
Processing Record 18 of Set 1 | geraldton
Processing Record 19 of Set 1 | jamestown
Processing Record 20 of Set 1 | mehamn
Processing Record 21 of Set 1 | saldanha
Processing Record 2

In [18]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Komsomolskiy,40.4272,71.7189,53.02,37,59,4.61,UZ,broken clouds,2022-03-03 15:21:35
1,Hilo,19.7297,-155.0900,72.52,93,0,4.00,US,clear sky,2022-03-03 15:23:56
2,Fortuna,40.5982,-124.1573,49.42,95,100,5.75,US,overcast clouds,2022-03-03 15:22:43
3,Thompson,55.7435,-97.8558,-9.24,77,0,12.66,CA,clear sky,2022-03-03 15:22:36
4,Meulaboh,4.1363,96.1285,80.06,79,99,2.24,ID,overcast clouds,2022-03-03 15:22:37
...,...,...,...,...,...,...,...,...,...,...
701,Gueret,46.1667,1.8667,51.24,70,98,2.71,FR,overcast clouds,2022-03-03 15:45:03
702,Paseh,-7.1026,107.7641,69.48,94,100,2.48,ID,overcast clouds,2022-03-03 15:45:03
703,Edd,13.9297,41.6925,89.37,31,10,24.94,ER,clear sky,2022-03-03 15:45:03
704,Cayenne,4.9333,-52.3333,81.09,84,75,8.05,GF,light rain,2022-03-03 15:43:56
